In [ ]:
!pip3 install pandas dash

In [9]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = df_data_1
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
@app.callback(
                Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value')
                )

def get_pie_chart(entered_site):
    filtered_df = spacex_df

    if entered_site == 'ALL':
        #g1 = filtered_df.groupby(["class"]).size().reset_index(name='count')
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total success launches by all sites')
        return fig
    else:
        filtered_df_2 =  filtered_df[filtered_df['Launch Site']==entered_site]
        g1 = filtered_df_2.groupby(["class"]).size().reset_index(name='count')
        this_title= 'Total success launches by ' + entered_site
        fig = px.pie(g1, values='count', names='class', title=this_title)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
                Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'),
                 Input(component_id="payload-slider", component_property="value")]
                )
def get_scatter_chart(entered_site, entered_payload):
    scatter_df = spacex_df
    low, high = entered_payload
    if entered_site == 'ALL':
        mask = (scatter_df['Payload Mass (kg)'] > low) & (scatter_df['Payload Mass (kg)'] < high)
        fig = px.scatter(scatter_df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig
    else:
        filtered_df_2 =  scatter_df[scatter_df['Launch Site']==entered_site]
        mask = (filtered_df_2['Payload Mass (kg)'] > low) & (filtered_df_2['Payload Mass (kg)'] < high)
        fig = px.scatter(filtered_df_2[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
